In [1]:
import visualize_wealth
import visualize_wealth.construct_portfolio as vwcp
import pandas
import datetime
from datetime import date
import numpy as np
import math

In [5]:
def simulate_portfolio(sheet_name, start_date='01/01/2000'):
    # Let's use the static_allocation provided in the panel from weight file.xlsx workbook
    f = pandas.ExcelFile('test_data/panel from weight file test.xlsx')
    static_alloc = f.parse(sheet_name, index_col = 0, header_col = 0)
    # Again, assume we don't have the prices and need to donwload them, use the fetch_data_for_initial_allocation_method
    price_panel  = vwcp.fetch_data_for_initial_allocation_method(static_alloc, start_date)
    # Construct the panel for the portoflio while determining the desired rebalance frequency
    panel        = vwcp.panel_from_initial_weights(weight_series = static_alloc, price_panel = price_panel, rebal_frequency = 'quarterly')
    # Construct the final portfolio with columns ['Open', 'Close']
    portfolio    = vwcp.pfp_from_weight_file(panel)
    return portfolio

def get_dt_object(obj):
    timestamp = "{}".format(obj)
    dt1 = datetime.datetime.strptime(timestamp, "%Y-%m-%d %H:%M:%S")
    return dt1

def get_time_in_year(portfolio):
    start_date = get_dt_object(portfolio.iloc[0].name)
    end_date = get_dt_object(portfolio.iloc[-1].name)
    print("start-date: {}, end-date: {}, ".format(start_date, end_date))
    return (end_date - start_date).days/365

def compute_avg_compund_return_rate(start_fund, final_fund, years):
    return (math.pow(10, math.log(final_fund/start_fund, 10)/years)-1)*100

def compute_avg_simple_return_rate(start_fund, final_fund, years):
    return ((final_fund - start_fund) * 100 / (years * start_fund)) 

def compute_avg_annual_return_rate(portfolio):
    initial_fund = portfolio.iloc[0,]['Open']
    final_fund = portfolio.iloc[-1,]['Close']
    years = get_time_in_year(portfolio)
    avg_simple_rate_per_year = compute_avg_simple_return_rate(initial_fund, final_fund, years)
    avg_comp_rate_per_year   = compute_avg_compund_return_rate(initial_fund, final_fund, years)
    print("initial-fund: {}, final-fund: {}".format(initial_fund, final_fund))
    print ("Average annual rate of return (simple interest): {}".format(avg_simple_rate_per_year))
    print ("Average annual rate of return (compound interest): {}".format(avg_comp_rate_per_year))
    
def compute_last_known_ticker_price(price_panel, ticker):
    last_known_price = price_panel[ticker].iloc[-1]
    return (last_known_price['Open'] + last_known_price['Close'])/2

def balance_portfolio(sheet_name, curr_total_investment, additional_amount_to_invest, start_date='01/01/2000'):
    f = pandas.ExcelFile('test_data/panel from weight file test.xlsx')
    static_alloc = f.parse(sheet_name, index_col = 0, header_col = 0)

    next_total_investment = curr_total_investment + additional_amount_to_invest
    ticker_count = len(static_alloc)

    # Add a new column with the price of the ticker symbols
    price = []
    price_panel  = vwcp.fetch_data_for_initial_allocation_method(static_alloc, start_date)
    for idx in range(ticker_count):
         price.append(compute_last_known_ticker_price(price_panel, static_alloc.index[idx]))
    static_alloc['Price'] = price

    # Add the current value column
    static_alloc['Current Value'] = static_alloc['Allocation'] * curr_total_investment
    # Add the next value column
    static_alloc['Next Value'] = static_alloc['Next Allocation'] * next_total_investment
    static_alloc['Diff Value'] = static_alloc['Next Value'] - static_alloc['Current Value']
    # Add shares to buy
    static_alloc['Shares to buy'] = np.floor(static_alloc['Diff Value'] / static_alloc['Price'])
    # Actual Next Value
    static_alloc['Actual Next Value'] = static_alloc['Shares to buy'] * static_alloc['Price']
    # Residual
    static_alloc['Residual'] = static_alloc['Diff Value'] - static_alloc['Actual Next Value']
    # Add the sum row at the bottom
    sum_row = list(map((lambda x: sum(static_alloc[x])), static_alloc.columns.values))
    static_alloc.loc[ticker_count] = np.array(sum_row)
    return static_alloc

Rebalance portfolio

In [6]:
sheet_name = 'static_allocation_arsinha'
balance_portfolio(sheet_name, 0, 10000)

,Allocation,Next Allocation,Price,Current Value,Next Value,Diff Value,Shares to buy,Actual Next Value,Residual
Ticker,,,,,,,,,
IEMG,0.17,0.17,47.820000,0.0,1700.0,1700.0,35.0,1673.699983,26.300018
IVV,0.13,0.13,237.319999,0.0,1300.0,1300.0,5.0,1186.599997,113.400003
IVE,0.12,0.12,104.180001,0.0,1200.0,1200.0,11.0,1145.980006,54.019994
IEFA,0.10,0.10,57.880001,0.0,1000.0,1000.0,17.0,983.960017,16.039983
FNDF,0.08,0.08,26.754999,0.0,800.0,800.0,29.0,775.894971,24.105029
SCHH,0.07,0.07,40.660000,0.0,700.0,700.0,17.0,691.220000,8.780000
STIP,0.07,0.07,101.430001,0.0,700.0,700.0,6.0,608.580003,91.419997
AGG,0.04,0.04,108.465000,0.0,400.0,400.0,3.0,325.395000,74.605000
SCZ,0.06,0.06,54.059999,0.0,600.0,600.0,11.0,594.659994,5.340006


Back-test some allocations

In [8]:
print("======= Some random allocation ========")
portfolio = simulate_portfolio('static_allocation')    
compute_avg_annual_return_rate(portfolio)
print("======= All SPY allocation ========")
portfolio = simulate_portfolio('all_spy', start_date='12/12/2007')    
compute_avg_annual_return_rate(portfolio)

======= Some random allocation ========
start-date: 2007-12-12 00:00:00, end-date: 2017-03-24 00:00:00, 
initial-fund: 1007.8859300415243, final-fund: 1458.892234015897
Average annual rate of return (simple interest): 4.817973378659043
Average annual rate of return (compound interest): 4.062202628524125
======= All SPY allocation ========
start-date: 2007-12-12 00:00:00, end-date: 2017-03-24 00:00:00, 
initial-fund: 1011.3142067119974, final-fund: 1911.2402649826022
Average annual rate of return (simple interest): 9.581067184315904
Average annual rate of return (compound interest): 7.093478131656861


In [109]:
print("======= Some random allocation ========")
portfolio = simulate_portfolio('allocation1')    
compute_avg_annual_return_rate(portfolio)

======= Some random allocation ========
start-date: 2000-01-03 00:00:00, end-date: 2017-03-24 00:00:00, 
initial-fund: 1003.3677336890592, final-fund: 4030.85296522
Average annual rate of return (simple interest): 17.509112026167994
Average annual rate of return (compound interest): 8.404077660055265
